In [43]:
import pandas as pd
import numpy as np
from glob import glob
import random
from PIL import Image 
from time import time

In [32]:
meta_data = pd.read_csv('../../../local_files/Projects/Instagram_Likes_Local/data/meta_data.csv')

In [38]:
meta_data = meta_data[meta_data['username'].notnull()]
usernames = meta_data.username.unique()
meta_data['rating'] = 0
meta_data.head(2)

,caption,comments,image_urls,likes,pro,user_url,username,insta_id,is_verified,following,full_name,followers,posts,pic_path,checksum,rating
0,{ Timeless Beauty } • Breathtaking place to fe...,196.0,http://scontent.cdninstagram.com/t51.2885-15/s...,25518.0,1.0,http://www.imgrum.org/user/brahmino/1918184,brahmino,1918184.0,False,341.0,Simone Bramante,830765.0,1194.0,2299079fa299a4b2ad784ebe588243865f45791c.jpg,6d382a79e19ef8eef973b715d63180aa,0
1,{ Golden Morning } • Happy to wake up early an...,298.0,http://scontent.cdninstagram.com/t51.2885-15/s...,28436.0,1.0,http://www.imgrum.org/user/brahmino/1918184,brahmino,1918184.0,False,341.0,Simone Bramante,830765.0,1194.0,d92fc8d70b8e71fa9ea53ced58dcd2db3cbe95db.jpg,3757b15a6fe424b2c543f07e48d87d60,0


In [39]:
def getRating(username, df):
    df = df[df['username']==username]
    ind = df.index.values
    df = df.likes.values
    increments = int(len(df)*.025)
    ratings = []
    for i in range(len(df)): 
        start = max(0, i-increments)
        end = min(len(df)-1, i+increments)
        mean = np.mean(df[start:end])
        std = np.std(df[start:end])
        if df[i]>=(mean + std*.3):
            ratings.append(1)
        elif df[i]<=(mean - std*.5):
            ratings.append(0)
        else:
            ratings.append(-1)
        zipped = list(zip(ind,ratings))
        rate = pd.DataFrame(zipped,columns=['ind', 'updated_rating'])
        rate.set_index(['ind'], inplace=True)
    return rate

In [45]:
def ratePhotos(usernames, df):
    t0 = time()
    for i in usernames:
        new_rated = getRating(i, df)
        df.rating.update(new_rated.updated_rating)
    print("done in %0.3fs." % (time() - t0))
    return df

In [46]:
meta_data.rating.value_counts()

0    160381
Name: rating, dtype: int64

In [47]:
meta_data = ratePhotos(usernames, meta_data)

/Users/bauer/anaconda3/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/bauer/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/bauer/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/bauer/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/bauer/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


done in 186.180s.


In [48]:
meta_data.rating.value_counts()

-1    55325
 0    54437
 1    50619
Name: rating, dtype: int64

In [55]:
meta_in = glob("../../../local_files/Projects/Instagram_Likes_Local/InstaImages/photographers_random/thumbs/big/*.jpg")
meta_out = '../../../local_files/Projects/Instagram_Likes_Local/InstaImages/photographers_random/photo_rand/'

In [56]:
size = 224, 224

def sortPhotos(df, input_path, output_path):
    for c in input_path:
        img = Image.open(c)
        img.thumbnail(size, Image.ANTIALIAS)
    #     img = Image.fromarray(img)
        rating = df.loc[df['pic_path']==c[-44:]]['rating']
        pro = df.loc[df['pic_path']==c[-44:]]['pro']
        if (rating+pro == 2).any():
            if random.random() > .25:
                if random.random() > .25:
                    img.save(output_path+'train/photo'+c[-45:])
                else: 
                    img.save(output_path+'validation/rand'+c[-45:])
            else:
                img.save(output_path+'test/photo'+c[-45:])
        elif (rating+pro ==0).any():
            if random.random() > .25:
                if random.random() > .3:
                    img.save(output_path+'train/rand'+c[-45:])
                else:
                    img.save(output_path+'validation/rand'+c[-45:])
            else: 
                img.save(output_path+'test/rand'+c[-45:])

In [57]:
sortPhotos(meta_data, meta_in, meta_out)